In [1]:
# 라이브러리 불러오기기
from datasets import load_dataset
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
print("✅ 라이브러리 불러오기 완료")

c:\Users\sksxk\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 라이브러리 불러오기 완료


In [2]:
# 불러온 데이터셋 확인
dataset = load_dataset("neuralsorcerer/student-performance")
df = dataset["train"].to_pandas()
df.head()

,Age,Grade,Gender,Race,SES_Quartile,ParentalEducation,SchoolType,Locale,TestScore_Math,TestScore_Reading,...,GPA,AttendanceRate,StudyHours,InternetAccess,Extracurricular,PartTimeJob,ParentSupport,Romantic,FreeTime,GoOut
0,17,12,Female,White,3,HS,Public,Suburban,70.258235,79.665185,...,2.743879,0.906809,1.089243,1,1,0,0,0,2,2
1,18,12,Female,White,2,HS,Public,Suburban,67.876546,59.611505,...,2.587700,0.843739,0.879911,1,1,0,1,0,5,2
2,16,11,Female,White,2,HS,Public,City,62.070623,70.577443,...,2.426294,0.850923,1.003442,1,1,0,1,1,5,1
3,14,9,Female,White,4,Bachelors+,Public,Town,79.909312,82.924450,...,3.558240,0.946786,1.814810,1,1,0,1,0,4,2
4,16,11,Female,Two-or-more,4,SomeCollege,Public,Rural,90.562294,95.352336,...,3.896884,0.988036,1.157241,1,1,0,0,0,3,4


In [ ]:

# ✅ 데이터 로딩 및 저장/재사용 로직
# 처음 실행 시 HuggingFace에서 불러오고, 이후는 저장된 파일에서 불러옴

import os
import pandas as pd
from datasets import load_dataset

data_path = "./data/student_data.pkl"

if not os.path.exists(data_path):
    print("🔄 HuggingFace에서 처음으로 데이터를 불러옵니다...")
    dataset = load_dataset("neuralsorcerer/student-performance")
    df = dataset["train"].to_pandas()

    # 필요한 열만 추출하고 결측치 제거
    cols = ["Gender", "ParentalEducation", "TestScore_Math", "TestScore_Reading", "TestScore_Science"]
    df = df[cols].dropna()

    # 인코딩
    from sklearn.preprocessing import LabelEncoder
    le_gender = LabelEncoder()
    le_parent = LabelEncoder()
    df["Gender"] = le_gender.fit_transform(df["Gender"])
    df["ParentalEducation"] = le_parent.fit_transform(df["ParentalEducation"])

    # 저장
    df.to_pickle(data_path)
    print(f"✅ 전처리 완료. '{data_path}'에 저장되었습니다.")
else:
    print(f"📁 저장된 데이터셋을 불러옵니다: '{data_path}'")
    df = pd.read_pickle(data_path)

df.head()


In [ ]:
le_gender = LabelEncoder()
le_parent = LabelEncoder()
df["Gender"] = le_gender.fit_transform(df["Gender"])
df["ParentalEducation"] = le_parent.fit_transform(df["ParentalEducation"])
df.head()

In [ ]:
df.groupby("Gender")[["TestScore_Math", "TestScore_Reading", "TestScore_Science"]].mean()

In [ ]:
sns.boxplot(x="Gender", y="TestScore_Math", data=df)
plt.title("성별에 따른 수학 점수 분포")
plt.xlabel("Gender (0: Female, 1: Male)")
plt.ylabel("TestScore_Math")
plt.tight_layout()
plt.show()

In [ ]:
X = df[["Gender", "ParentalEducation"]]
y = df["TestScore_Math"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("R²:", r2_score(y_test, y_pred))

In [ ]:
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--")
plt.xlabel("실제 수학 점수")
plt.ylabel("예측된 수학 점수")
plt.title("예측 vs 실제 수학 점수")
plt.tight_layout()
plt.show()